# Propensity to Buy Scoring Feed With Biased Data

This notebook can be scheduled to provide a continuous flow of data to your Propensity model. Paste your Watson Machine Learning credentials below, and verify that the value of the **DEPLOYMENT_NAME** variable matches the name of your deployed Propensity model. You can also set the maximum and minimum number of records that will be scored per run.

In [ ]:
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1

In [ ]:
WML_CREDENTIALS = {Insert WML Credentials Here}

In [ ]:
DEPLOYMENT_NAME = "Propensity Model"
MIN_RECORDS = 1000
MAX_RECORDS = 1000

In [ ]:
import json, random
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [ ]:
!rm df_payload_biased.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/df_payload_biased.json

In [ ]:
wml_deployments = wml_client.deployments.get_details()
scoring_url = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        scoring_url = deployment['entity']['scoring_url']
        break
    
print("Scoring URL: {}".format(scoring_url))

In [ ]:
try:
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file found')
    
except:
    !wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/df_payload_biased.json
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file downloaded')

In [ ]:
fields = scoring_data['fields']
values = []
for _ in range(0, random.randint(MIN_RECORDS, MAX_RECORDS)):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)
print(scoring_response)